# Capston Assignment

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [4]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [5]:
table_rows = table.find_all('tr')

#table_rows
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#print(df.head(5))
#print('***')
#print(df.tail(5))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [7]:
df.shape

(180, 3)

In [8]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [9]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [10]:
df1 = df.reset_index()

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [12]:
df1.shape

(103, 4)

In [13]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

In [14]:
df2.info()
df2.shape

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


(103, 2)

In [15]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df3.info()
df3.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

In [21]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [28]:
from geopy.geocoders import Nominatim
import certifi

geolocator = Nominatim(user_agent="jypiter_notbook")
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [30]:
import pandas as pd
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="jypiter_notbook")

In [31]:
df_geopy1 = df3

In [33]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="jypiter_notbook")

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [34]:
df_geopy1 = df3
df3.shape

(103, 4)

In [35]:
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [36]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)
#df_geopy1
# code holds true up until i=102
df_geopy1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [37]:
df_geopy1.shape



(103, 4)

In [38]:
df_geopy1.to_csv('geopy1.csv')

In [40]:
from  geopy.geocoders import Nominatim

location = geolocator.geocode("M1G, Scarborough, Woburn")


In [41]:
pip install geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [42]:
df3.to_csv('geopy.csv')
import csv

with open('geopy.csv') as csvfile:
     reader = csv.DictReader(csvfile)

In [44]:
from  geopy.geocoders import Nominatim

location = geolocator.geocode("M1B Scarborough Rouge,Malvern")

In [45]:
from  geopy.geocoders import Nominatim
location = geolocator.geocode("Toronto, Highland Creek")

In [46]:
from  geopy.geocoders import Nominatim
location = geolocator.geocode("Toronto, Morningside")

In [47]:
import numpy as np
import csv


PostalCode = None
Borough = None
Neighbourhood = None
latData = None
longData = None

LAT_Woburn = 43.7598243
LONG_Woburn = -79.2252908
LAT_Malvern = 43.8091955
LONG_Malvern = -79.2217008
LAT_Highland_Creek = 43.7901172
LONG_Highland_Creek = -79.1733344
LAT_Morningside = 43.7826012
LONG_Morningside = -79.2049579

 
PostalCode = np.array(['M1H','M1B','M1C','M1G '])
Borough = np.array(['Scarborough','Scarborough','Scarborough','Scarborough'])
Neighbourhood = np.array(['Woburn','Malvern','Highland_Creek','Morningside'])
latData = np.array([43.7598243,43.8091955, 43.7901172 , 43.7826012])
longData = np.array([-79.2252908,-79.2217008,-79.1733344, -79.2049579 ])

with open('data.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow('ABXYZ')
    for a,b,x,y,z in np.nditer([ PostalCode.T, Borough.T, Neighbourhood.T, latData.T, longData.T], order='C'):
        writer.writerow([a,b,x,y,z])    

In [48]:
import csv

with open('data.csv') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row['A'], row['B'],row['X'], row['Y'], row['Z'])

M1H Scarborough Woburn 43.7598243 -79.2252908
M1B Scarborough Malvern 43.8091955 -79.2217008
M1C Scarborough Highland_Creek 43.7901172 -79.1733344
M1G  Scarborough Morningside 43.7826012 -79.2049579


In [49]:
pd.read_csv('data.csv') 

,A,B,X,Y,Z
0,M1H,Scarborough,Woburn,43.759824,-79.225291
1,M1B,Scarborough,Malvern,43.809196,-79.221701
2,M1C,Scarborough,Highland_Creek,43.790117,-79.173334
3,M1G,Scarborough,Morningside,43.782601,-79.204958


In [50]:
import pandas, os
#os.listdir()
df_geopy=df3
import geopy
type(df_geopy)
df_geopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 4 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: object(4)
memory usage: 4.0+ KB


In [118]:
pip install geopy


The following command must be run outside of the IPython shell:

    $ pip install geopy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [144]:
from geopy.geocoders import Nominatim
print('Nominatim imported')

Nominatim imported


In [145]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M1B,Scarborough,"Malvern, Rouge","M1B,Scarborough,Malvern, Rouge",None
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C,Scarborough,Rouge Hill, Port Union, Highla...",None
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E,Scarborough,Guildwood, Morningside, West Hill",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


In [146]:
from geopy.geocoders import Nominatim

nom = Nominatim(user_agent="abc")
n = nom.geocode('M1B, Scarborough, Rouge,Malvern')
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n,n2)

None None


In [147]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)




In [148]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Malvern, Rouge","M1B,Scarborough,Malvern, Rouge",None,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C,Scarborough,Rouge Hill, Port Union, Highla...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E,Scarborough,Guildwood, Morningside, West Hill",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN


In [149]:
df_geopy.to_csv('geo_loc_py.csv')

In [150]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

The latitude of M1B,Scarborough,Malvern, Rouge is nan and its longitude is nan


In [151]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data2 = pd.read_csv("geopy.csv") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address
0,0,M1B,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [160]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-10c9ba49-97d7-48ed-be8d-efd29b6e8851',
    'IBM_API_KEY_ID': 'O2dIDo8QRRUtACv_yylFxhK9qlf9TOKdcL0Y6c3LOCvM',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'assignmentcapstonno2-donotdelete-pr-chtsd1uapygagp',
    'FILE': 'Geospatial_Coordinates.csv'
}
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e2a3ea17e20e49c186567359f1919d5a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='O2dIDo8QRRUtACv_yylFxhK9qlf9TOKdcL0Y6c3LOCvM',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_e2a3ea17e20e49c186567359f1919d5a.get_object(Bucket='assignmentcapstonno2-donotdelete-pr-chtsd1uapygagp',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data3 = pd.read_csv(body)
data3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [161]:
data3.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [162]:
data1 = pd.merge(data3, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data1.head()

,PostalCode,Latitude,Longitude,Unnamed: 0,Borough,Neighbourhood,address
0,M1B,43.806686,-79.194353,0,Scarborough,"Malvern, Rouge","M1B, Scarborough, Malvern, Rouge"
1,M1C,43.784535,-79.160497,1,Scarborough,"Rouge Hill, Port Union, Highland Creek","M1C, Scarborough, Rouge Hill, Port Union, High..."
2,M1E,43.763573,-79.188711,2,Scarborough,"Guildwood, Morningside, West Hill","M1E, Scarborough, Guildwood, Morningside, West..."
3,M1G,43.770992,-79.216917,3,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,43.773136,-79.239476,4,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [163]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 7 columns):
PostalCode       103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
Unnamed: 0       103 non-null int64
Borough          103 non-null object
Neighbourhood    103 non-null object
address          103 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 6.4+ KB


In [164]:
cols = data1.columns.tolist()
cols

['PostalCode',
 'Latitude',
 'Longitude',
 'Unnamed: 0',
 'Borough',
 'Neighbourhood',
 'address']

In [165]:
new_column_order = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
new_column_order

['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [166]:
data1 = data1[new_column_order]

In [167]:
sorted_df = data1.sort_values([ 'Neighbourhood', 'Latitude'], ascending=[True, True])

In [168]:
sorted_df.reset_index(inplace=True)

In [169]:
sorted_cols =sorted_df.columns.tolist()

In [170]:
new_column_order2 = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude']
sorted_dataframe = sorted_df[new_column_order]
sorted_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1S,Scarborough,Agincourt,43.794200,-79.262029
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M3H,NorthYork,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,M2K,NorthYork,Bayview Village,43.786947,-79.385975
4,M5M,NorthYork,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


In [171]:
sorted_dataframe.to_csv('sorted_geoloc.csv')